In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests

In [3]:
url = 'https://www.redfin.com/county/2918/UT/Utah-County'

In [4]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get(url)

In [7]:
driver.quit()


In [5]:
import time

prices = []
beds = []
baths = []
square_footage = []
address = []
current_page = 1
pagination = driver.find_element(By.CLASS_NAME, 'PageNumbers')
last_page = pagination.find_elements(By.CLASS_NAME, 'ButtonLabel')[-1].text
while current_page <= int(last_page):
    container = driver.find_element(By.CLASS_NAME, 'HomeCardsContainer')
    all_homes = container.find_elements(By.CLASS_NAME, 'HomeCardContainer')
    for home in all_homes:
        try:
            price = home.find_element(By.CLASS_NAME, 'bp-Homecard__Price--value').text
            bed = home.find_element(By.CLASS_NAME, 'bp-Homecard__Stats--beds').text
            bath = home.find_element(By.CLASS_NAME, 'bp-Homecard__Stats--baths').text
            sqft = home.find_element(By.CLASS_NAME, 'bp-Homecard__LockedStat--value').text
            add = home.find_element(By.CLASS_NAME, 'bp-Homecard__Address').text
            key = home.find_element(By.CLASS_NAME, 'KeyFactsExtension').text
        except:
            price = None
            bed = None
            bath = None
        prices.append(price)
        beds.append(bed)
        baths.append(bath)
        square_footage.append(sqft)
        address.append(add)
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, '.PageArrow__direction--next')
        next_button.click()
        current_page += 1
        time.sleep(2)
    except:
        pass

In [6]:
df = pd.DataFrame({'Price': prices, 'Beds': beds, 'Baths': baths, 'Sq_Footage': square_footage, 'Address': address})
print(df.head(2))
print(df.tail(2))
df

      Price    Beds      Baths Sq_Footage  \
0  $575,000  4 beds  2.5 baths      3,609   
1  $775,000  6 beds  3.5 baths      4,148   

                                          Address  
0  979 W Minuteman Ln, Saratoga Springs, UT 84045  
1           274 E Wood Thrush Cv, Salem, UT 84653  
          Price    Beds      Baths Sq_Footage  \
385  $2,300,000  7 beds  4.5 baths      5,685   
386  $1,795,000  8 beds  4.5 baths      6,553   

                                               Address  
385      11838 N Westfield Cove Dr, Highland, UT 84003  
386  1852 S Centennial Blvd E, Saratoga Springs, UT...  


,Price,Beds,Baths,Sq_Footage,Address
0,"$575,000",4 beds,2.5 baths,"3,609","979 W Minuteman Ln, Saratoga Springs, UT 84045"
1,"$775,000",6 beds,3.5 baths,"4,148","274 E Wood Thrush Cv, Salem, UT 84653"
2,None,None,None,"4,148","274 E Wood Thrush Cv, Salem, UT 84653"
3,"$667,647",4 beds,3 baths,"3,949","5430 N Pony Rider Way #422, Eagle Mountain, UT..."
4,"$379,990",3 beds,2 baths,"1,399","1577 S 3740 E #1305, Spanish Fork, UT 84660"
...,...,...,...,...,...
382,"$699,000",5 beds,3 baths,"3,371","1479 S 1960 E, Spanish Fork, UT 84660"
383,"$279,900",2 beds,1 bath,905,"1140 W 950 N Unit D205, Orem, UT 84057"
384,"$489,000",4 beds,2 baths,"2,320","683 S 560 E, Orem, UT 84097"
385,"$2,300,000",7 beds,4.5 baths,"5,685","11838 N Westfield Cove Dr, Highland, UT 84003"
